# Лабораторная работа №3 (Проведение исследований с решающим деревом)

## Создание бейзлайна и оценка качества

### Обучение моделей из sklearn для выбранных наборов данных

#### Импорт библиотек

In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from scipy.spatial import distance
import numpy as np
from collections import Counter

#### Подготовка данных

##### Классификация (Rice MSC Dataset)

**Загрузка данных:**

In [125]:
rice_data = pd.read_csv('Rice_MSC_Dataset.csv')

X_rice = rice_data.iloc[:, :-1]  # Признаки
y_rice = rice_data.iloc[:, -1]   # Метки классов

label_encoder = LabelEncoder()
y_rice_encoded = label_encoder.fit_transform(y_rice)

**Разделение на тренировочный и тестовый наборы:**

In [126]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)

**Замена пропущенных значений**

In [127]:
def simple_imputer_df(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return pd.DataFrame(imputer.fit_transform(x_train)), pd.DataFrame(imputer.transform(x_test))

In [128]:
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)

**Масштабирование данных**

In [129]:
def scaling(x_train, x_test):
    scaler = StandardScaler()
    return scaler.fit_transform(x_train), scaler.transform(x_test)

In [130]:
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

##### Регрессия (Gold Price Regression)

**Загрузка данных:**

In [96]:
gold_data = pd.read_csv('financial_regression.csv')

X_gold = gold_data.iloc[:, 1:-1]  # Признаки
y_gold = gold_data.iloc[:, -1]   # Целевая переменная

**Разделение на тренировочный и тестовый наборы:**

In [97]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)

**Замена пропущенных значений:**

In [98]:
def simple_imputer(x_train, x_test):
    imputer = SimpleImputer(strategy='mean')
    return imputer.fit_transform(x_train), imputer.transform(x_test)

In [99]:
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)

**Удаление строк с пропущенными значениями:**

In [100]:
def delete_none(x_train, y_train, x_test, y_test):
    mask = ~np.isnan(y_train)
    mask_test = ~np.isnan(y_test)
    return x_train[mask], y_train[mask], x_test[mask_test], y_test[mask_test]

In [101]:
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

#### Обучение моделей

##### Классификация

In [131]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_rice, y_train_rice)

DecisionTreeClassifier(random_state=42)

##### Регрессия

In [103]:
reg = DecisionTreeRegressor(random_state=42)
reg.fit(X_train_gold, y_train_gold)

DecisionTreeRegressor(random_state=42)

### Оценка качества моделей по выбранным метрикам

##### Классификация

In [116]:
def quality_eval_classification(X_test, y_test, model):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    return accuracy, f1

In [132]:
accuracy_origin, f1_origin = quality_eval_classification(X_test_rice, y_test_rice, clf)

Accuracy: 0.9961333333333333
F1 Score: 0.9961328742196198


##### Регрессия

In [133]:
def quality_eval_regression(X_test, y_test, model):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Mean Absolute Error: {mae}")
    print(f"R-squared: {r2}")
    return mae, r2

In [135]:
mae_origin, r2_origin = quality_eval_regression(X_test_gold, y_test_gold, reg)

Mean Absolute Error: 3645966.6435246994
R-squared: 0.1661606200483663


## Улучшение бейзлайна

### Гипотезы

1. Подбор гиперпараметров:
   - Использование Grid Search с кросс-валидацией для оптимизации модели.

### Проверка гипотез

#### 1. Подбор гиперпараметров

##### Классификация

Подготовка данных:

In [143]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

Определение параметров для перебора:

In [148]:
param_grid = {
   'max_depth': [3, 5, 10, None],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4],
}

Настройка и обучение GridSearchCV:

In [149]:
grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_rice, y_train_rice)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [3, 5, 10, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_weighted')

Вывод лучших параметров и результата:

In [150]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5}
Best cross-validation accuracy:  0.9963506488864955


Оценка качества модели:

In [151]:
quality_eval_classification(X_test_rice, y_test_rice, grid_search.best_estimator_)

Accuracy: 0.9958666666666667
F1 Score: 0.9958670780767213


(0.9958666666666667, 0.9958670780767213)

##### Регрессия

Подготовка данных:

In [174]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

Определение параметров для перебора:

In [175]:
param_grid = {
   'max_depth': [3, 5, 10, None],
   'min_samples_split': [2, 5, 10],
   'min_samples_leaf': [1, 2, 4],
}

Настройка и обучение GridSearchCV:

In [176]:
grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid, cv=5, scoring='r2')
grid_search.fit(X_train_gold, y_train_gold)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [3, 5, 10, None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='r2')

Вывод лучших параметров и результата:

In [177]:
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

Best parameters found:  {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best cross-validation accuracy:  0.43376301381918925


Оценка модели с лучшими параметрами на тестовом наборе:

In [159]:
quality_eval_regression(X_test_gold, y_test_gold, grid_search.best_estimator_)

Mean Absolute Error: 3032799.598401824
R-squared: 0.4704552864463365


(3032799.598401824, 0.4704552864463365)

### Формирование улучшенного бейзлайна

##### Классификация

Формирование улучшенного бейзлайна:

In [178]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)
                                   
clf = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_leaf=1, min_samples_split=5)

##### Регрессия

In [179]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

reg = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_leaf=4, min_samples_split=2)

### Обучение модели с улучшенным бейзлайном

##### Классификация

Обучение модели:

In [180]:
clf.fit(X_train_rice, y_train_rice)

DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=42)

##### Регрессия

Обучение модели:

In [181]:
reg.fit(X_train_gold, y_train_gold)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, random_state=42)

### Оценка качества модели с улучшенным бейзлайном

##### Классификация

Оценка качества модели:

In [182]:
accuracy_improved, f1_improved = quality_eval_classification(X_test_rice, y_test_rice, clf)

Accuracy: 0.9958666666666667
F1 Score: 0.9958670780767213


##### Регрессия

Оценка качества модели:

In [183]:
mae_improved, r2_improved = quality_eval_regression(X_test_gold, y_test_gold, reg)

Mean Absolute Error: 8226987.037383177
R-squared: -1.4486458799625517


### Сравнение результатов первоначального бейзлайна с улучшенным бейзлайном

##### Классификация

Видим, что результаты немного ухудшились:

In [184]:
print(f"Accuracy difference: {accuracy_improved-accuracy_origin}")
print(f"F1 Score difference: {f1_improved-f1_origin}")

Accuracy difference: -0.0002666666666666373
F1 Score difference: -0.0002657961428985667


##### Регрессия

Видим, что результаты сильно ухудшились:

In [185]:
print(f"Mean Absolute Error difference: {mae_origin-mae_improved}")
print(f"R-squared difference: {r2_improved-r2_origin}")

Mean Absolute Error difference: -4581020.3938584775
R-squared difference: -1.614806500010918


### Выводы

Улучшенный бейзлайн оказался хуже базового на тестовой выборке. Хотя на обучающей выборке было всё лучше. Скорей всего произошло переобучение.

## Имплементация алгоритма машинного обучения

### Самостоятельная имплементация

Решающее дерево строится за счет рекурсивного деления данных на основе критерия, такого как энтропия или индекс Джини для классификации, и среднеквадратичной ошибки для регрессии.

##### Классификация

In [187]:
class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None

    def fit(self, X, y):
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _gini(self, y):
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

    def _best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
        best_gini = 1.0 - sum((num / m) ** 2 for num in num_parent)
        best_idx, best_thr = None, None

        for idx in range(n):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes_
            num_right = num_parent.copy()

            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum((x / i) ** 2 for x in num_left)
                gini_right = 1.0 - sum((x / (m - i)) ** 2 for x in num_right)
                gini = (i * gini_left + (m - i) * gini_right) / m

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2

        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = {"gini": self._gini(y), "num_samples": len(y),
                "num_samples_per_class": num_samples_per_class,
                "predicted_class": predicted_class}

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node["feature_index"] = idx
                node["threshold"] = thr
                node["left"] = self._grow_tree(X_left, y_left, depth + 1)
                node["right"] = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while "threshold" in node:
            if inputs[node["feature_index"]] < node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["predicted_class"]

##### Регрессия

In [193]:
class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None

    def fit(self, X, y):
        self.tree_ = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _mse(self, y):
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _best_split(self, X, y):
        m, n = X.shape
        if m <= 1:
            return None, None

        best_mse = self._mse(y)
        best_idx, best_thr = None, None

        for idx in range(n):
            thresholds, values = zip(*sorted(zip(X[:, idx], y)))
            left_values = []
            right_values = list(values)

            for i in range(1, m):
                value = values[i - 1]
                left_values.append(value)
                right_values.pop(0)
                
                if i < self.min_samples_split or len(right_values) < self.min_samples_split:
                    continue

                mse_left = self._mse(left_values)
                mse_right = self._mse(right_values)
                mse = (i * mse_left + (m - i) * mse_right) / m

                if thresholds[i] == thresholds[i - 1]:
                    continue

                if mse < best_mse:
                    best_mse = mse
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2

        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        node = {"value": np.mean(y)}

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node["feature_index"] = idx
                node["threshold"] = thr
                node["left"] = self._grow_tree(X_left, y_left, depth + 1)
                node["right"] = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree_
        while "threshold" in node:
            if inputs[node["feature_index"]] < node["threshold"]:
                node = node["left"]
            else:
                node = node["right"]
        return node["value"]

### Обучение имплементированной модели

##### Классификация

Подготовка данных:

In [191]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

clf = DecisionTreeClassifierCustom(max_depth=5)

Обучение модели:

In [192]:
clf.fit(X_train_rice, y_train_rice)

##### Регрессия

Подготовка данных:

In [194]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

reg = DecisionTreeRegressorCustom(max_depth=5)

Обучение модели:

In [195]:
reg.fit(X_train_gold, y_train_gold)

### Оценка качества имплементированных моделей

##### Классификация

Оценка качества модели:

In [196]:
accuracy_my, f1_my = quality_eval_classification(X_test_rice, y_test_rice, clf)

Accuracy: 0.9936666666666667
F1 Score: 0.9936697410476582


##### Регрессия

Оценка качества модели:

In [197]:
mae_my, r2_my = quality_eval_regression(X_test_gold, y_test_gold, reg)

Mean Absolute Error: 2981618.7395315934
R-squared: 0.5162302518023489


### Сравнение результатов первоначального бейзлайна и имплементированных моделей

##### Классификация

Видим, что результаты имплементированной модели немного хуже:

In [199]:
print(f"Accuracy difference: {accuracy_my-accuracy_origin}")
print(f"F1 Score difference: {f1_my-f1_origin}")

Accuracy difference: -0.002466666666666617
F1 Score difference: -0.0024631331719616734


##### Регрессия

Видим, что результаты имплементированной модели хуже:

In [201]:
print(f"Mean Absolute Error difference: {mae_my-mae_origin}")
print(f"R-squared difference: {r2_my-r2_origin}")

Mean Absolute Error difference: -664347.903993106
R-squared difference: 0.35006963175398265


### Выводы

Оценка качества наших имплементированных моделей оказалась хуже.
Можно сделать следующие выводы:
1. Предобработка данных важна: Масштабирование и нормализация данных могут значительно повлиять на качество модели.
2. Оптимизация гиперпараметров: Правильная настройка параметров, таких как темп обучения и количество итераций, необходима для достижения оптимальной производительности.
3. Регуляризация: Помогает избежать переобучения и может улучшить обобщающую способность модели.
4. Алгоритмическая точность: Правильная реализация алгоритма и внимание к численным ошибкам критичны для качества модели.
5. Используйте проверенные решения: Изучение и сравнение с библиотечными реализациями может помочь выявить слабые места вашей модели.
6. Тщательно изучайте данные: Соблюдение чистоты, отсутствие пропусков и ошибок в данных — важный аспект построения качественной модели.

### Добавление техник из улучшенного бейзлайна

##### Классификация

In [210]:
X_train_rice, X_test_rice, y_train_rice, y_test_rice = train_test_split(X_rice, y_rice_encoded, test_size=0.2, random_state=42)
X_train_rice, X_test_rice = simple_imputer_df(X_train_rice, X_test_rice)
X_train_rice, X_test_rice = scaling(X_train_rice, X_test_rice)

clf = DecisionTreeClassifierCustom(max_depth=10, min_samples_split=5)

##### Регрессия

Формирование улучшенного бейзлайна:

In [211]:
X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(X_gold, y_gold, test_size=0.2, random_state=42)
X_train_gold, X_test_gold = simple_imputer(X_train_gold, X_test_gold)
X_train_gold, X_test_gold = scaling(X_train_gold, X_test_gold)
X_train_gold, y_train_gold, X_test_gold, y_test_gold = delete_none(X_train_gold, y_train_gold, X_test_gold, y_test_gold)

reg = DecisionTreeRegressorCustom(max_depth=5, min_samples_split=2)

### Обучение на улучшенном бейзлайне

##### Классификация

In [212]:
clf.fit(X_train_rice, y_train_rice)

##### Регрессия

In [213]:
reg.fit(X_train_gold, y_train_gold)

### Оценка качества моделей на улучшенном бейзлайне

##### Классификация

In [214]:
accuracy_my_improved, f1_my_improved = quality_eval_classification(X_test_rice, y_test_rice, clf)

Accuracy: 0.9964
F1 Score: 0.9964006808545692


##### Регрессия

In [207]:
mae_my_impoved, r2_my_impoved = quality_eval_regression(X_test_gold, y_test_gold, lin_reg)

Mean Absolute Error: 2736143.926621333
R-squared: 0.5192060277943523


### Сравнение результатов

##### Классификация

In [208]:
print(f"Accuracy difference: {accuracy_my_improved-accuracy_improved}")
print(f"F1 Score difference: {f1_my_improved-f1_improved}")

Accuracy difference: 0.0005333333333332746
F1 Score difference: 0.0005336027778479746


##### Регрессия

In [209]:
print(f"Mean Absolute Error difference: {mae_my_impoved-mae_improved}")
print(f"R-squared difference: {r2_my_impoved-r2_improved}")

Mean Absolute Error difference: -5490843.110761844
R-squared difference: 1.9678519077569039


### Выводы

Результаты получились лучше, особенно для задачи регрессии. И лучше базового бейзлайна. До сих пор не очень понятно, почему для улучшенного бейзлайна для задачи регрессии на готовой имплементации была такая ошибка, но на собственной имплементации таких проблем нет.